In [23]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!pip install numpy
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool
from spektral.transforms.normalize_adj import NormalizeAdj

In [26]:
!rm -r YALE-RESEARCH-WORK

In [27]:
!git clone https://github.com/harshit5674/YALE-RESEARCH-WORK.git

Cloning into 'YALE-RESEARCH-WORK'...
remote: Enumerating objects: 345089, done.
remote: Total 345089 (delta 0), reused 0 (delta 0), pack-reused 345089
Receiving objects: 100% (345089/345089), 128.51 MiB | 27.05 MiB/s, done.
Resolving deltas: 100% (345032/345032), done.
Checking out files: 100% (70013/70013), done.


In [28]:
!ls

sample_data  YALE-RESEARCH-WORK


In [29]:
path="YALE-RESEARCH-WORK/dataset_automata"

In [30]:
class MyDataset(Dataset):
    
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
      
    def read(self):
      output=[]
      for i in range(0,70000):
         data = np.load(os.path.join(path, f'graph_{i}.npz'))
         output.append(Graph(x=data['x'], a=data['a'], e=data['e'],y=data['y']))
        
      return output

In [31]:
class MyDataset1(Dataset):
    
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
      
    def read(self):
      output=[]
      for i in range(0,70000):
         data = np.load(os.path.join(path, f'graph_{i}.npz'))
         output.append(Graph(x=data['x'], a=data['a'], y=data['y']))
        
      return output

In [32]:
import os

In [36]:
dataset = MyDataset(transforms=NormalizeAdj()) 
idxs = np.random.permutation(len(dataset))
split = int(0.85 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

In [37]:
path="YALE-RESEARCH-WORK/dataset_automata"

In [38]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool
################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = ECCConv(32, activation="relu")
        self.conv2 = ECCConv(32, activation="relu")
        self.conv3=ECCConv(32,activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        x = self.conv3([x,a,e])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0

ov_count=0
ov_total_count=0
for batch in loader_te:
  
  inputs, target = batch
  predictions = model(inputs, training=False)
  loss += loss_fn(target, predictions)
  for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
       # print(er[-1])
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Loss: 64.33604431152344
Loss: 58.70257568359375
Loss: 57.860450744628906
Loss: 57.064918518066406
Loss: 56.09347152709961
Loss: 55.27485656738281
Loss: 54.91847229003906
Loss: 54.5775146484375
Loss: 54.4722900390625
Loss: 54.287750244140625
Testing model
Done. Test loss: 53.94426727294922
2303
3380
7772
10500


In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv
from spektral.models.gcn import GCN

################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(32, attn_heads=3, concat_heads=True, dropout_rate=0.5, return_attn_coef=False, add_self_loops=True, activation="relu", use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', attn_kernel_initializer='glorot_uniform', kernel_regularizer=None, bias_regularizer=None, attn_kernel_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, attn_kernel_constraint=None)
        self.conv2 =GATConv(32, attn_heads=3, concat_heads=True, dropout_rate=0.5, return_attn_coef=False, add_self_loops=True, activation="relu", use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', attn_kernel_initializer='glorot_uniform')
        self.conv3 =GATConv(32, attn_heads=3, concat_heads=True, dropout_rate=0.5, return_attn_coef=False, add_self_loops=True, activation="relu", use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', attn_kernel_initializer='glorot_uniform')
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_11/gat_conv_7/Reshape_5:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_11/gat_conv_7/Reshape_4:0", shape=(None, 3), dtype=float32), dense_shape=Tensor("gradient_tape/net_11/gat_conv_7/Cast_1:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_11/gat_conv_7/Reshape_8:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_11/gat_conv_7/Reshape_7:0", shape=(None, 3), dtype=float32), dense_shape=Tensor("gradient_tape/net_11/gat_conv_7/Cast_2:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a l

Loss: 71.6545181274414
Loss: 68.27639770507812
Loss: 67.35057067871094
Loss: 67.00225830078125
Loss: 66.67135620117188
Loss: 66.2906723022461
Loss: 66.27021026611328
Loss: 65.97164154052734
Loss: 65.71961212158203
Loss: 65.64010620117188
Testing model
Done. Test loss: 70.5650405883789
1113
3380
6976
10500


In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool,MessagePassing
################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = ECCConv(32, activation="relu")
        self.conv2 = ECCConv(32, activation="relu")
        self.conv3=MessagePassing(aggregate='sum')
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        x = self.conv3([x,a,e])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0

ov_count=0
ov_total_count=0
for batch in loader_te:
  
  inputs, target = batch
  predictions = model(inputs, training=False)
  loss += loss_fn(target, predictions)
  for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
       # print(er[-1])
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

Loss: 65.84223175048828
Loss: 60.28924560546875
Loss: 59.26241683959961
Loss: 58.956634521484375
Loss: 58.6509895324707
Loss: 58.60116195678711
Loss: 58.41450500488281
Loss: 58.276371002197266
Loss: 58.2286262512207
Loss: 58.14802169799805
Testing model
Done. Test loss: 57.23370361328125
1921
3380
7474
10500


In [41]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv
from spektral.models.gcn import GCN

################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(32, attn_heads=3, concat_heads=True, dropout_rate=0.5, return_attn_coef=False, add_self_loops=True, activation="relu", use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', attn_kernel_initializer='glorot_uniform', kernel_regularizer=None, bias_regularizer=None, attn_kernel_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, attn_kernel_constraint=None)
        self.conv2 =GATConv(32, attn_heads=3, concat_heads=True, dropout_rate=0.5, return_attn_coef=False, add_self_loops=True, activation="relu", use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', attn_kernel_initializer='glorot_uniform')
        self.conv3=MessagePassing(aggregate='sum')

        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_13/gat_conv_9/Reshape_5:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_13/gat_conv_9/Reshape_4:0", shape=(None, 3), dtype=float32), dense_shape=Tensor("gradient_tape/net_13/gat_conv_9/Cast_1:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_13/gat_conv_9/Reshape_8:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_13/gat_conv_9/Reshape_7:0", shape=(None, 3), dtype=float32), dense_shape=Tensor("gradient_tape/net_13/gat_conv_9/Cast_2:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a l

Loss: 71.04652404785156
Loss: 68.45233154296875
Loss: 67.75459289550781
Loss: 67.03807830810547
Loss: 66.7838134765625
Loss: 66.44432830810547
Loss: 65.97178649902344
Loss: 65.88166046142578
Loss: 65.78783416748047
Loss: 65.5538558959961
Testing model
Done. Test loss: 67.02734375
1773
3380
6617
10500


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=AGNNConv(trainable=True, aggregate='sum', activation=None)
        self.conv2=AGNNConv(trainable=True, aggregate='sum', activation=None)
        self.conv3=AGNNConv(trainable=True, aggregate='sum', activation=None)


        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_14/agnn_conv_7/Reshape_4:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_14/agnn_conv_7/Reshape_3:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/net_14/agnn_conv_7/Cast_1:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_14/agnn_conv_7/Reshape_7:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_14/agnn_conv_7/Reshape_6:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/net_14/agnn_conv_7/Cast_2:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a

Loss: 97.63720703125
Loss: 76.5283432006836
Loss: 73.88966369628906
Loss: 72.96376037597656
Loss: 72.45005798339844
Loss: 72.14883422851562
Loss: 71.93587493896484
Loss: 71.77703094482422
Loss: 71.61277770996094
Loss: 71.48017120361328
Testing model
Done. Test loss: 70.46073913574219
934
3380
6691
10500


In [43]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=AGNNConv(trainable=True, aggregate='sum', activation=None)
        self.conv2=AGNNConv(trainable=True, aggregate='sum', activation=None)
        self.conv3=MessagePassing(aggregate='sum')


        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_15/agnn_conv_9/Reshape_4:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_15/agnn_conv_9/Reshape_3:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/net_15/agnn_conv_9/Cast_1:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/net_15/agnn_conv_9/Reshape_7:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/net_15/agnn_conv_9/Reshape_6:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/net_15/agnn_conv_9/Cast_2:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a

Loss: 90.60151672363281
Loss: 75.89082336425781
Loss: 74.1084976196289
Loss: 73.11143493652344
Loss: 72.56303405761719
Loss: 72.25262451171875
Loss: 72.0198745727539
Loss: 71.82147979736328
Loss: 71.64606475830078
Loss: 71.48455810546875
Testing model
Done. Test loss: 70.55004119873047
964
3380
6751
10500


In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv,GeneralConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=GeneralConv(channels=256, batch_norm=True, dropout=0.0, aggregate='sum', activation='prelu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv2=GeneralConv(channels=256, batch_norm=True, dropout=0.0, aggregate='sum', activation='prelu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv3=GeneralConv(channels=256, batch_norm=True, dropout=0.0, aggregate='sum', activation='prelu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

        

        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

Loss: 66.09622192382812
Loss: 64.6649169921875
Loss: 64.37018585205078
Loss: 64.12459564208984
Loss: 63.964515686035156
Loss: 63.688297271728516
Loss: 63.611759185791016
Loss: 63.49766159057617
Loss: 63.413818359375
Loss: 63.29599380493164
Testing model
Done. Test loss: 62.629600524902344
1432
3380
7011
10500


In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv,GeneralConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split

loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=GeneralConv(channels=256, batch_norm=True, dropout=0.0, aggregate='sum', activation='prelu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv2=GeneralConv(channels=256, batch_norm=True, dropout=0.0, aggregate='sum', activation='prelu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv3=MessagePassing(aggregate='sum')

        

        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

Loss: 69.73238372802734
Loss: 66.16064453125
Loss: 65.68498229980469
Loss: 65.2615737915039
Loss: 64.96328735351562
Loss: 64.62937927246094
Loss: 64.33808898925781
Loss: 64.05093383789062
Loss: 63.940956115722656
Loss: 63.80311584472656
Testing model
Done. Test loss: 63.41973876953125
1722
3380
6848
10500


In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv,GeneralConv,GraphSageConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=GraphSageConv(channels=256, aggregate='mean', activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv2=GraphSageConv(channels=256, aggregate='mean', activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv3=GraphSageConv(channels=256, aggregate='mean', activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)


        

        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

Loss: 67.85474395751953
Loss: 64.70655059814453
Loss: 64.209228515625
Loss: 63.847625732421875
Loss: 63.48682403564453
Loss: 63.2465934753418
Loss: 62.93763732910156
Loss: 62.77790832519531
Loss: 62.532508850097656
Loss: 62.342811584472656
Testing model
Done. Test loss: 61.57862854003906
1184
3380
7242
10500


In [47]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv,GeneralConv,GraphSageConv
from spektral.models.gcn import GCN


################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target


loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1=GraphSageConv(channels=256, aggregate='mean', activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv2=GraphSageConv(channels=256, aggregate='mean', activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
        self.conv3=MessagePassing(aggregate='sum')


        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

Loss: 66.97750091552734
Loss: 65.23843383789062
Loss: 64.8147201538086
Loss: 64.45862579345703
Loss: 64.13648986816406
Loss: 63.899169921875
Loss: 63.64555740356445
Loss: 63.49897766113281
Loss: 63.214271545410156
Loss: 63.00121307373047
Testing model
Done. Test loss: 61.96047592163086
1354
3380
7138
10500


In [33]:
dataset = MyDataset1(transforms=NormalizeAdj()) 
idxs = np.random.permutation(len(dataset))
split = int(0.85 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GATConv, MessagePassing, AGNNConv,GeneralConv,GraphSageConv
from spektral.models.general_gnn import GeneralGNN



################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10 # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split

loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Build model
################################################################################

model = GeneralGNN(output=44, activation=None, hidden=256, message_passing=4, pre_process=2, post_process=2, connectivity='cat', batch_norm=True, dropout=0.0, aggregate='sum', hidden_activation='prelu', pool='sum')

optimizer = Adam(learning_rate)
loss_fn = MeanSquaredError()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
count=0
total_count=0
ov_count=0
ov_total_count=0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    for i in range(predictions.shape[0]):
      er=predictions[i]
      we=target[i]
      ok=0
      if er[-1]<=40:
        ok=0
      else:
        ok=100
      if ok==we[-1] and ok==100:
        count=count+1
      if we[-1]==100:
        total_count=total_count+1
      if we[-1]==ok:
        ov_count=ov_count+1
      ov_total_count=ov_total_count+1
    loss += loss_fn(target, predictions)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))
print(count)
print(total_count)
print(ov_count)
print(ov_total_count)

/usr/local/lib/python3.8/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Loss: 127.97563171386719
Loss: 105.89288330078125
Loss: 91.37091064453125
Loss: 82.13446044921875
Loss: 76.80769348144531
Loss: 73.62044525146484
Loss: 71.12588500976562
Loss: 68.99361419677734
Loss: 67.03250122070312
Loss: 65.2644271850586
Testing model
Done. Test loss: 59.2122917175293
1232
3435
7657
10500
